In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import os
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import math
from tensorflow.keras.layers import GRU, LSTM, Conv1D, Dense, BatchNormalization, Dropout, Bidirectional, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import initializers

In [3]:
train_file = [os.path.join('sentiment_analysis', i) for i in os.listdir('sentiment_analysis') if 'train' in i][0]
test_file = [os.path.join('sentiment_analysis', i) for i in os.listdir('sentiment_analysis') if 'test' in i][0]

In [4]:
train_raw = pd.read_csv(train_file)
test_raw = pd.read_csv(test_file)

In [5]:
train_raw

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...
...,...,...,...
7915,7916,0,Live out loud #lol #liveoutloud #selfie #smile...
7916,7917,0,We would like to wish you an amazing day! Make...
7917,7918,0,Helping my lovely 90 year old neighbor with he...
7918,7919,0,Finally got my #smart #pocket #wifi stay conne...


In [6]:
def clean_non_word(file):
    
    for i in range(len(file)):
        
        file.iloc[i, 2] = re.sub('[^A-Za-z\s]', '', file.iloc[i, 2])
    
    return file

In [7]:
train_no_nonword = train_raw.copy()
train_no_nonword = clean_non_word(train_no_nonword)
train_no_nonword

,id,label,tweet
0,1,0,fingerprint Pregnancy Test httpsgooglhMfQV and...
1,2,0,Finally a transparant silicon case Thanks to ...
2,3,0,We love this Would you go talk makememories un...
3,4,0,Im wired I know Im George I was made that way ...
4,5,1,What amazing service Apple wont even talk to m...
...,...,...,...
7915,7916,0,Live out loud lol liveoutloud selfie smile son...
7916,7917,0,We would like to wish you an amazing day Make ...
7917,7918,0,Helping my lovely year old neighbor with her ...
7918,7919,0,Finally got my smart pocket wifi stay connecte...


In [8]:
def lower_case(file):
    
    for i in range(len(file)):
        
        file.iloc[i, 2] = file.iloc[i, 2].lower()
    
    return file

In [9]:
train_no_nonword = lower_case(train_no_nonword)
train_no_nonword

,id,label,tweet
0,1,0,fingerprint pregnancy test httpsgooglhmfqv and...
1,2,0,finally a transparant silicon case thanks to ...
2,3,0,we love this would you go talk makememories un...
3,4,0,im wired i know im george i was made that way ...
4,5,1,what amazing service apple wont even talk to m...
...,...,...,...
7915,7916,0,live out loud lol liveoutloud selfie smile son...
7916,7917,0,we would like to wish you an amazing day make ...
7917,7918,0,helping my lovely year old neighbor with her ...
7918,7919,0,finally got my smart pocket wifi stay connecte...


In [11]:
word_freq_dict = {}
def create_word_freq_dict(file):
    
    global word_freq_dict
        
    for i in range(len(file)):
            
        for word in file.iloc[i, 2].split():
                
            if word not in word_freq_dict.keys():
                
                word_freq_dict[word] = 1
                
            else:
                    
                word_freq_dict[word] += 1

In [12]:
create_word_freq_dict(train_no_nonword)

In [13]:
ordered_freq_dict = {k: v for k, v in sorted(word_freq_dict.items(), key = lambda item: item[1])}

In [14]:
final_freq_dict = {k:v for k, v in word_freq_dict.items() if v > 1}
len(final_freq_dict)

6169

In [15]:
stopword = stopwords.words('english')
word_list = [k for k in word_freq_dict.keys() if k not in stopword and 'http' not in k]
word_index = {v:k+1 for k, v in enumerate(word_list)}
word_index['oov'] = len(word_index) + 1
word_index

{'fingerprint': 1,
 'pregnancy': 2,
 'test': 3,
 'android': 4,
 'apps': 5,
 'beautiful': 6,
 'cute': 7,
 'health': 8,
 'igers': 9,
 'iphoneonly': 10,
 'iphonesia': 11,
 'iphone': 12,
 'finally': 13,
 'transparant': 14,
 'silicon': 15,
 'case': 16,
 'thanks': 17,
 'uncle': 18,
 'yay': 19,
 'sony': 20,
 'xperia': 21,
 'sonyexperias': 22,
 'love': 23,
 'would': 24,
 'go': 25,
 'talk': 26,
 'makememories': 27,
 'unplug': 28,
 'relax': 29,
 'smartphone': 30,
 'wifi': 31,
 'connect': 32,
 'im': 33,
 'wired': 34,
 'know': 35,
 'george': 36,
 'made': 37,
 'way': 38,
 'daventry': 39,
 'home': 40,
 'amazing': 41,
 'service': 42,
 'apple': 43,
 'wont': 44,
 'even': 45,
 'question': 46,
 'unless': 47,
 'pay': 48,
 'stupid': 49,
 'support': 50,
 'software': 51,
 'update': 52,
 'fucked': 53,
 'phone': 54,
 'big': 55,
 'time': 56,
 'iphones': 57,
 'happy': 58,
 'us': 59,
 'instapic': 60,
 'instadaily': 61,
 'xperiaz': 62,
 'new': 63,
 'type': 64,
 'c': 65,
 'charger': 66,
 'cable': 67,
 'uk': 68,
 'b

In [16]:
len(word_index)

17317

In [17]:
def longest_corpus(data):
    
    max_len = 0
    
    for i in range(len(data)):
        
        temp_len = len(data.iloc[i, 2].split())
        
        if temp_len > max_len:
            
            max_len = temp_len
    
    return max_len

In [18]:
max_pad = longest_corpus(train_no_nonword)
max_pad

52

In [19]:
def create_training_set(file, padding = 'none', max_pad = 50):
    
    train = []
    
    global word_index
    
    for i in range(len(file)):
        
        sample_train = []
        
        for word in file.iloc[i, 2].split():
            
            if word not in stopword:
            
                try:
                    sample_train.append(word_index[word])

                except KeyError:
                    sample_train.append(len(word_index) + 1)
        
        if padding != 'none':
                    
            if len(sample_train) < 52:

                if padding == 'post':

                    for i in range(int(max_pad) - len(sample_train)):

                        sample_train.append(0)

                else:

                    for i in range(int(max_pad) - len(sample_train)):

                        sample_train.insert(0, 0)
                

        train.append([sample_train, file.iloc[i, 1]])
    
    return train

In [20]:
train_full = create_training_set(train_no_nonword, padding = 'post', max_pad = max_pad)
train_full[0]

[[1,
  2,
  3,
  0,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 1]

In [21]:
split_point = math.ceil(len(train_no_nonword) * 0.8)

In [22]:
train_full = np.array(train_full)

In [23]:
sample_index = np.random.randint(0, len(train_no_nonword), size = len(train_no_nonword))
train_df = train_full[sample_index[0:split_point]]
val_df = train_full[sample_index[split_point:]]
print('Length of training set: ', len(train_df))

Length of training set:  6336


In [24]:
def split_corpus_label(data):
    
    x = []
    y = []
    
    for i in range(len(data)):
        
        x.append(data[i][0])
        y.append(data[i][1])
    
    return np.array(x), np.array(y)

In [25]:
train_x, train_y = split_corpus_label(train_df)
val_x, val_y = split_corpus_label(val_df)
train_x[0]

array([  24,  162, 1308,   41,  219,  107,  114, 1309, 1310, 1311,  697,
         12, 1312,  452,  270, 5691,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0])

In [26]:
input_dim = len(word_index)
output_dim = 100
input_length = max_pad

In [27]:
model = Sequential()
model.add(Embedding(input_dim = input_dim, output_dim = output_dim, mask_zero = True, 
                    input_length = input_length, embeddings_initializer=initializers.glorot_uniform(seed=0)))

model.add(Conv1D(64, 8, input_shape = [None, 100], activation = 'relu', kernel_initializer=initializers.glorot_uniform(seed=0)))

model.add(Bidirectional(GRU(128, return_sequences = True, kernel_initializer=initializers.glorot_uniform(seed=0))))
model.add(Bidirectional(GRU(128, return_sequences = False, kernel_initializer=initializers.glorot_uniform(seed=0))))

model.add(Dense(256, kernel_initializer=initializers.glorot_uniform(seed=0), kernel_regularizer=tf.keras.regularizers.L1(0.01), activity_regularizer=tf.keras.regularizers.L2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(1, kernel_initializer=initializers.glorot_uniform(seed=0), kernel_regularizer=tf.keras.regularizers.L1(0.01), activity_regularizer=tf.keras.regularizers.L2(0.01)))

In [28]:
model.compile(optimizer = tf.keras.optimizers.Adam(lr = 5e-4), loss = 'binary_crossentropy', metrics = ['acc'])

In [31]:
history = model.fit(train_x, train_y, batch_size = 32, validation_data = (val_x, val_y), epochs = 75)

Epoch 1/75
198/198 [==============================] - 6s 31ms/step - loss: 1.1162 - acc: 0.9871 - val_loss: 1.5215 - val_acc: 0.9028
Epoch 2/75
198/198 [==============================] - 6s 29ms/step - loss: 0.9615 - acc: 0.9874 - val_loss: 1.4216 - val_acc: 0.9047
Epoch 3/75
198/198 [==============================] - 6s 29ms/step - loss: 0.8327 - acc: 0.9871 - val_loss: 1.3002 - val_acc: 0.9066
Epoch 4/75
198/198 [==============================] - 6s 29ms/step - loss: 0.7227 - acc: 0.9877 - val_loss: 1.2433 - val_acc: 0.9059
Epoch 5/75
198/198 [==============================] - 6s 29ms/step - loss: 0.6377 - acc: 0.9880 - val_loss: 1.2103 - val_acc: 0.9053
Epoch 6/75
198/198 [==============================] - 6s 29ms/step - loss: 0.5729 - acc: 0.9882 - val_loss: 1.1959 - val_acc: 0.9028
Epoch 7/75
198/198 [==============================] - 6s 29ms/step - loss: 0.5244 - acc: 0.9886 - val_loss: 1.1832 - val_acc: 0.9053
Epoch 8/75
198/198 [==============================] - 6s 29ms/step - 

In [32]:
input1 = tf.keras.layers.Input(shape = (52,))
embed = Embedding(input_dim = input_dim, output_dim = output_dim, mask_zero = True, 
                  input_length = input_length, embeddings_initializer=initializers.glorot_uniform(seed=0))(input1)

conv1 = Conv1D(64, 8, input_shape = [None, 100], activation = 'relu', kernel_initializer=initializers.glorot_uniform(seed=0))(embed)

output1, hidden_forward, hidden_backward = Bidirectional(
    GRU(128, return_sequences = True, return_state = True, kernel_initializer=initializers.glorot_uniform(seed=0)))(conv1)

gru2 = Bidirectional(
    GRU(128, return_sequences = False, kernel_initializer=initializers.glorot_uniform(seed=0)))(output1, 
                                                                                                initial_state = 
                                                                                                [hidden_forward, hidden_backward])

dense1 = Dense(256, kernel_initializer= initializers.glorot_uniform(seed=0), 
               kernel_regularizer=tf.keras.regularizers.L1(0.01), activity_regularizer=tf.keras.regularizers.L2(0.01))(gru2)
dense1 = Dropout(0.5)(dense1)
output = Dense(1, kernel_initializer= initializers.glorot_uniform(seed=0), 
               kernel_regularizer=tf.keras.regularizers.L1(0.01), activity_regularizer=tf.keras.regularizers.L2(0.01))(dense1)

state_model = tf.keras.models.Model(inputs = input1, outputs = output)

In [33]:
state_model.compile(optimizer = tf.keras.optimizers.Adam(lr = 5e-4), loss = 'binary_crossentropy', metrics = ['acc'])

In [34]:
history_statemodel = state_model.fit(train_x, train_y, batch_size = 32, validation_data = (val_x, val_y), epochs = 75)

Epoch 1/75
198/198 [==============================] - 7s 34ms/step - loss: 14.5582 - acc: 0.7884 - val_loss: 1.0994 - val_acc: 0.8813
Epoch 2/75
198/198 [==============================] - 6s 29ms/step - loss: 0.5588 - acc: 0.9416 - val_loss: 0.5624 - val_acc: 0.9261
Epoch 3/75
198/198 [==============================] - 6s 29ms/step - loss: 0.3796 - acc: 0.9785 - val_loss: 0.6997 - val_acc: 0.9356
Epoch 4/75
198/198 [==============================] - 6s 29ms/step - loss: 0.3222 - acc: 0.9882 - val_loss: 0.7085 - val_acc: 0.9306
Epoch 5/75
198/198 [==============================] - 6s 30ms/step - loss: 0.3093 - acc: 0.9888 - val_loss: 0.7878 - val_acc: 0.9287
Epoch 6/75
198/198 [==============================] - 6s 29ms/step - loss: 0.3016 - acc: 0.9883 - val_loss: 0.8399 - val_acc: 0.9293
Epoch 7/75
198/198 [==============================] - 6s 29ms/step - loss: 0.3023 - acc: 0.9883 - val_loss: 0.7656 - val_acc: 0.9306
Epoch 8/75
198/198 [==============================] - 6s 29ms/step -

In [36]:
history_statemodel.model.layers[1].weights

[<tf.Variable 'embedding_2/embeddings:0' shape=(17317, 100) dtype=float32, numpy=
 array([[-0.08968937,  0.08269832,  0.06013974, ..., -0.0033683 ,
          0.03669983, -0.08458614],
        [-0.01123074,  0.01817808, -0.01911442, ...,  0.0026936 ,
         -0.00221379, -0.0047959 ],
        [ 0.02828863,  0.03257012, -0.0433582 , ...,  0.01739149,
         -0.00377206, -0.02932386],
        ...,
        [ 0.02923309,  0.00359678, -0.04217791, ...,  0.03803669,
         -0.01304831, -0.01093927],
        [ 0.01023096,  0.00147674,  0.01376358, ..., -0.01292893,
         -0.00534817,  0.01604604],
        [ 0.0139632 , -0.00550926, -0.00546545, ...,  0.00499245,
         -0.0019279 ,  0.01020234]], dtype=float32)>]